In [2]:
import FinanceDataReader as fdr
import datetime

start_date = datetime.datetime.now() - datetime.timedelta(days=365*20)
# 현재 시간으로부터 20년 전 날짜 계산


df_kospi = fdr.DataReader('KS11', start_date).add_suffix('_kospi')
df_kosdaq = fdr.DataReader('KQ11', start_date).add_suffix('_kosdaq')
df_dow = fdr.DataReader('DJI', start_date).add_suffix('_dow')
df_nasdaq = fdr.DataReader('IXIC', start_date).add_suffix('_nasdaq')
df_snp500 = fdr.DataReader('US500', start_date).add_suffix('_s&p500')

In [4]:
import pandas as pd
import numpy as np

In [5]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

In [6]:
df_kospi

,Open_kospi,High_kospi,Low_kospi,Close_kospi,Adj Close_kospi,Volume_kospi
Date,,,,,,
2003-03-07,547.890015,555.380005,544.090027,546.020020,546.020020,635100.0
2003-03-10,545.539978,548.289978,541.289978,544.239990,544.239990,607100.0
2003-03-11,531.780029,538.619995,530.549988,532.530029,532.530029,657900.0
2003-03-12,531.169983,534.760010,524.530029,531.809998,531.809998,793800.0
2003-03-13,530.859985,531.780029,514.429993,531.780029,531.780029,679700.0
...,...,...,...,...,...,...
2023-02-22,2430.870117,2435.979980,2416.159912,2417.679932,2417.679932,426700.0
2023-02-23,2430.010010,2449.620117,2422.620117,2439.090088,2439.090088,400300.0
2023-02-24,2442.489990,2452.969971,2421.469971,2423.610107,2423.610107,369200.0


In [7]:
df_heelim = fdr.DataReader('037440', start_date).add_suffix('_heelim')
df_aps = fdr.DataReader('054620', start_date).add_suffix('_aps_holdings')
data_aps = pd.merge(df_kospi, df_kosdaq, on='Date').merge(df_dow, on='Date').merge(df_nasdaq, on='Date').merge(df_snp500, on='Date').merge(df_aps, on='Date')
data_heelim = pd.merge(df_kospi, df_kosdaq, on='Date').merge(df_dow, on='Date').merge(df_nasdaq, on='Date').merge(df_snp500, on='Date').merge(df_heelim, on='Date')

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [16]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)


In [24]:
#input_size = 784
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [ ]:
train_dataset = datasets.MNIST(root='dataset/', train=True,  transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='dataset/', train=False,  transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [28]:


# Initialize network
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to coda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data')
    else:
        print('checking accuracy on test data')
    
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

            print(f'Got {num_correct} / {num_samples} with accuracy \
                {float(num_correct) / float(num_samples) * 100:.2f}')
    
    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

100%|██████████| 9912422/9912422 [00:02<00:00, 4633808.99it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 7382721.47it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 3576523.55it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 12354428.51it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



RuntimeError: input.size(-1) must be equal to input_size. Expected 784, got 28